In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
def cargar_datos_video(ruta_datos):
    datos = []
    etiquetas = []
    categorias = os.listdir(ruta_datos)
    categoria_a_numero = {categoria: i for i, categoria in enumerate(categorias)}
    
    for categoria in categorias:
        ruta_categoria = os.path.join(ruta_datos, categoria)
        archivos_categoria = os.listdir(ruta_categoria)

        for archivo in archivos_categoria:
            ruta_archivo = os.path.join(ruta_categoria, archivo)
            cap = cv2.VideoCapture(ruta_archivo)
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                datos.append(frame)
                etiquetas.append(categoria_a_numero[categoria])
            cap.release()

    # Convertir listas de Python a arrays numpy
    datos = np.array(datos)
    etiquetas = np.array(etiquetas)

    return datos, etiquetas

In [ ]:
def cargar_datos_numpy(ruta_dataset):
    X = []
    y = []
    categorias = os.listdir(ruta_dataset)

    max_longitud = 0  # Variable para almacenar la longitud máxima encontrada

    for categoria in categorias:
        ruta_categoria = os.path.join(ruta_dataset, categoria)
        archivos = os.listdir(ruta_categoria)

        for archivo in archivos:
            ruta_archivo = os.path.join(ruta_categoria, archivo)
            datos = np.load(ruta_archivo)

            # Obtener la longitud de la matriz actual y actualizar la longitud máxima si es necesario
            longitud_actual = datos.shape[0]
            max_longitud = max(max_longitud, longitud_actual)

            X.append(datos)
            y.append(categoria)

    # Rellenar las matrices más cortas con ceros para que todas tengan la misma longitud
    for i in range(len(X)):
        longitud_actual = X[i].shape[0]
        if longitud_actual < max_longitud:
            # Crear una nueva matriz con ceros y copiar los datos existentes
            nueva_matriz = np.zeros((max_longitud, X[i].shape[1]))
            nueva_matriz[:longitud_actual, :] = X[i]
            X[i] = nueva_matriz

    return np.array(X), np.array(y)



In [ ]:
# Función para preparar los datos para la CNN
def preparar_datos(X, y):
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)  # Agregar una dimensión para los canales
    X = X / 255.0  # Normalizar
    y_numeros = {etiqueta: i for i, etiqueta in enumerate(np.unique(y))}
    y_encoded = np.array([y_numeros[etiqueta] for etiqueta in y])
    y_encoded = to_categorical(y_encoded)
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test


In [ ]:
# Obtener la ruta del directorio de datos
ruta_actual = os.getcwd()
ruta_raiz = os.path.dirname(os.path.dirname(ruta_actual))
ruta_dataset = os.path.join(ruta_raiz, 'Data_Set')
ruta_datos = os.path.join(ruta_dataset, 'numpy_procesado')


# Cargar y preparar el dataset
X, y = cargar_datos_numpy(os.path.join(ruta_dataset, 'numpy_procesado'))
X_train, X_test, y_train, y_test = preparar_datos(X, y)

# Definir arquitectura de la CNN
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))